# Fintech Churn predictive model
The following project aims at predicting the churn of users in the 30 days following his activation day (first transaction). We understand churn as no transaction made.
To do so, only user information obtained up to that 1st completed transaction will be used. 
This means that, from the moment a user made his 1st completed transaction, we will be able to predict whether a user will transact again in the next 30 days or not.

# Import libraries

In [1]:
# Librerías clásicas
import numpy as np
import pandas as pd
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import MinMaxScaler

# Models
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
#from xgboost import cv
#from sklearn.model_selection import cross_val_score
#from sklearn import metrics
import pickle

# Librerías para conectarnos a George
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

# 1) Read data from queries (Date range: today -60d, today-30d)

## a) Get data from DBs

Here we defined three queries to our DB to retrieve the datasets.
- query_media_source
- query_oi_and_td
- query_ud_and_accounts

Also, a connection to our PostgreSQL was built in the conn object.

In [4]:
conn = getPostgreConn()
data1 = pd.read_sql(query_media_source, index_col="user_id", con=conn)
data2 = pd.read_sql(query_oi_and_td, index_col="user_id", con=conn)
data3 = pd.read_sql(text(query_ud_and_accounts), index_col="user_id", con=conn)
closePostgreConn(conn)

## b) Merge data into one dataset

In [5]:
data_test = data1.merge(data2, left_index=True, right_index=True).merge(data3, left_index=True, right_index=True)

# 2) Preprocessing

## a) Cast variables correctly

In [6]:
def string_to_datetime(data, cols):
    data[cols] = data[cols].apply(pd.to_datetime)
    return data
    
def bool_to_binary(data,cols):
    data[cols] = data[cols]*1
    return data

In [7]:
data_test = string_to_datetime(data_test,['fecha_registro','fecha_completo_datos','fecha_validado','fecha_1ra_cuenta_propia','fecha_1ra_transacion','fecha_1ra_transacion_completed'])
data_test = bool_to_binary(data_test,'transfer_between_own_accounts')

## b) Create label variable. This step is skipped because for new data we do not have the target variable.

In [8]:
# def create_label_variable(data,target,threshold=0):
#     data["label"] = 0
#     data.loc[data[target] > threshold,"label"] = 1
#     return data

In [9]:
# data_test = create_label_variable(data_test,"ret_30d",0)

## c) Discretize amount of first transaction.

In [10]:
# Creo una variable amt_bins para segmentar el aamount
data_test["amt_bins"] = data_test["amount"]
data_test.loc[(data_test["amt_bins"]>0)&(data_test["amt_bins"]<=100),"amt_bins"] = 100
data_test.loc[(data_test["amt_bins"]>100)&(data_test["amt_bins"]<=300),"amt_bins"] = 300
data_test.loc[(data_test["amt_bins"]>300)&(data_test["amt_bins"]<=500),"amt_bins"] = 500
data_test.loc[(data_test["amt_bins"]>500)&(data_test["amt_bins"]<=1000),"amt_bins"] = 1000
data_test.loc[(data_test["amt_bins"]>1000)&(data_test["amt_bins"]<=2000),"amt_bins"] = 2000
data_test.loc[(data_test["amt_bins"]>2000)&(data_test["amt_bins"]<=5000),"amt_bins"] = 5000
data_test.loc[data_test["amt_bins"]>5000,"amt_bins"] = 10000

## d) Calculate hours between onboarding steps.

In [11]:
#Calculamos la cantidad de horas entre cada paso del onboarding
data_test["hours_OBR-OBC"] = (data_test["fecha_completo_datos"]-data_test["fecha_registro"]).astype('timedelta64[s]')/(60*60)
data_test["hours_OBC-OBV"] = (data_test["fecha_validado"]-data_test["fecha_completo_datos"]).astype('timedelta64[s]')/(60*60)
data_test["hours_OBV-OB1Acc"] = (data_test["fecha_1ra_cuenta_propia"]-data_test["fecha_validado"]).astype('timedelta64[s]')/(60*60)
data_test["hours_OB1Acc-FTUc"] = (data_test["fecha_1ra_transacion_completed"]-data_test["fecha_1ra_cuenta_propia"]).astype('timedelta64[s]')/(60*60)

## e) Impute gender missing values

In [12]:
#Leo el dataset de nombres de usuarios
data_names = pd.read_csv("data\hombres_mujeres.csv")

#separo el dataset en uno de hombres y uno de mujeres
dataset_hombres = pd.DataFrame(data_names.loc[data_names["gender"]=="M"][["first_name","gender"]])
dataset_mujeres = pd.DataFrame(data_names.loc[data_names["gender"]=="F"][["first_name","gender"]])

#transformo cada dataset en un diccionario
dicc_hombres = dict(zip(dataset_hombres.first_name,dataset_hombres.gender))
dicc_mujeres = dict(zip(dataset_mujeres.first_name,dataset_mujeres.gender))

#imputo primero los datos faltantes que estén en el dataset de hombres y luego los que estén en el de mujeres
data_test.loc[data_test["gender"].isnull(),"gender"] = data_test.loc[data_test["gender"].isnull(),"first_name"].map(dicc_hombres)
data_test.loc[data_test["gender"].isnull(),"gender"] = data_test.loc[data_test["gender"].isnull(),"first_name"].map(dicc_mujeres)

## f) Get day and time of first transaction completed

In [13]:
def split_datetime_in_variables(data,variables):
    dataaux = data.copy()
    for i in variables:
        dataaux[i+'_day'] = dataaux[i].dt.day
        dataaux[i+'_hour'] = dataaux[i].dt.hour
    return dataaux

split_datetime_in_variables(data_test, ['fecha_1ra_transacion_completed'])

# Generamos variables para el día y la hora de activación
data_test['fecha_1ra_transacion_completed_day'] = data_test['fecha_1ra_transacion_completed'].dt.day
data_test['fecha_1ra_transacion_completed_hour'] = data_test['fecha_1ra_transacion_completed'].dt.hour

# Encoding con funciones trigonométricas
data_test['fecha_1ra_transacion_completed_hour_sin'] = np.sin(data_test.fecha_1ra_transacion_completed_hour*(2.*np.pi/24))
data_test['fecha_1ra_transacion_completed_hour_cos'] = np.cos(data_test.fecha_1ra_transacion_completed_hour*(2.*np.pi/24))
data_test['fecha_1ra_transacion_completed_day_sin'] = np.sin(data_test.fecha_1ra_transacion_completed_day*(2.*np.pi/31))
data_test['fecha_1ra_transacion_completed_day_cos'] = np.cos(data_test.fecha_1ra_transacion_completed_day*(2.*np.pi/31))

data_test.drop(columns=["fecha_1ra_transacion_completed_day","fecha_1ra_transacion_completed_hour"], inplace=True)

## g) Drop unnecessary variables

In [14]:
data_test.drop(columns=["fecha_registro","fecha_completo_datos","fecha_validado","fecha_1ra_cuenta_propia","fecha_1ra_transacion","fecha_1ra_transacion_completed",
"ret_30d","first_name"], inplace=True)

## h) Cluster banks by type

In [15]:
dict_bancos = {'BRUBANK':'banco_virtual',
                'BBVA':'banco_privado', 
               'BICA':'banco_privado', 
               'BIND':'banco_privado',
               'COINAG':'banco_privado', 
               'COLUMBIA':'banco_privado', 
               'COMAFI':'banco_privado',
               'CREDICOOP':'banco_privado',
               'DINO':'banco_privado', 
               'GALICIA':'banco_privado',
               'HIPOTECARIO':'banco_privado',
               'HSBC':'banco_privado', 
               'ICBC':'banco_privado', 
               'ITAU':'banco_privado',
               'MARIVA':'banco_privado',
               'MACRO':'banco_privado', 
               'MASVENTAS':'banco_privado', 
               'PATAGONIA':'banco_privado', 
               'PIANO':'banco_privado', 
               'ROELA':'banco_privado', 
               'SANTANDER':'banco_privado', 
               'SUPERVIELLE':'banco_privado',
               'NACION':'banco_nacional',
               'BANCOR':'banco_provincial', 
               'BSE':'banco_provincial', 
               'CHACO':'banco_provincial',
               'CHUBUT':'banco_provincial', 
               'CIUDAD':'banco_provincial', 
               'CORRIENTES':'banco_provincial', 
               'ENTRERIOS':'banco_provincial', 
               'FORMOSA':'banco_provincial', 
               'LAPAMPA':'banco_provincial', 
               'NEUQUEN':'banco_provincial',
               'PROVINCIA':'banco_provincial', 
               'RIOJA':'banco_provincial', 
               'ROSARIO':'banco_provincial', 
               'SANJUAN':'banco_provincial',
               'SANTACRUZ':'banco_provincial', 
               'SANTAFE':'banco_provincial',
               'TIERRADELFUEGO':'banco_provincial',
               'FISERV':'comercio',
               'MERCADOPAGO':'billetera',
               'NARANJAX':'banco_virtual',
               'NUBI':'banco_virtual',
               'OPENBANK':'banco_virtual',
               'REBANKING':'banco_virtual',
               'TAP':'billetera',
               'UALA':'billetera',
               'YACARE':'comercio'}

In [16]:
def group_banks_by_type(dataset,columns,dict_bancos):
    for i in range(len(columns)):
        dataset.loc[:,columns[i]] = dataset.loc[:,columns[i]].map(dict_bancos)

group_banks_by_type(data_test,["debit_account_bank","credit_account_bank"],dict_bancos)

## i) Group accounts by type

In [17]:
# pasamos los valores CA y CC a CBU
dict_cuentas = {'CA':'CBU',
                'CV':'CVU',
                'CC':'CBU' 
}

In [18]:
def group_accounts_by_type(dataset,columns,dict_cuentas):
    for i in range(len(columns)):
        dataset.loc[:,columns[i]] = dataset.loc[:,columns[i]].map(dict_cuentas)

group_accounts_by_type(data_test,["credit_account_type"],dict_cuentas)

## j) Calculate the logarithm of the amount of the first transaction completed

In [19]:
data_test["log_amount"] = np.log(data_test.amount)
data_test.drop(columns="amount", inplace=True)

## k) Group the lookup-type

In [20]:
dict_lookup = {'CBU':'CBU',
               'PHONE':'PHONE OR EMAIL',
               'EMAIL':'PHONE OR EMAIL'    
}

In [21]:
def group_lookups_by_type(dataset,columns,dict_lookup):
    for i in range(len(columns)):
        dataset.loc[:,columns[i]] = dataset.loc[:,columns[i]].map(dict_lookup)
        
group_lookups_by_type(data_test,["lookup_key_type"],dict_lookup)

## l) Discretize the number of attempts to the first transaction completed

In [22]:
# Binarizamos la variable
data_test.loc[data_test.cant_attempts==0,"cant_attempts"]=0
data_test.loc[data_test.cant_attempts>0,"cant_attempts"]=1

## m) Discretize gender

In [23]:
dict_gender = {'F':0,'M':1}

data_test.loc[:,'gender'] = data_test.loc[:,'gender'].map(dict_gender)

## n) Ordinal encoding of age group variable

In [24]:
dict_age = {'25-':0, '25-34':1, '35-45':2, '55+':3, '46-55':4}
data_test.loc[:,'age_group'] = data_test.loc[:,'age_group'].map(dict_age)

## o) Discretize the number of bank contacts

In [25]:
data_test.loc[data_test.cant_contactos_bancos >= 1,"cant_contactos_bancos"] = 1

## p) Discretize the number of other contacts

In [26]:
data_test.loc[data_test.cant_contactos_ank > 0,"cant_contactos_ank"] = 1

## q) Group number of accounts

In [27]:
data_test.loc[data_test["cantidad_de_cuentas"]>1,"cantidad_de_cuentas"]=2

## r) Group number of banks

In [28]:
data_test.loc[data_test["cantidad_de_bancos"]>1,"cantidad_de_bancos"]=2

## s) Group number of cbu and cvu accounts

In [29]:
data_test.loc[data_test["cantidad_cvu"]>0,"cantidad_cvu"]=1
data_test.loc[data_test["cantidad_cbu"]>1,"cantidad_cbu"]=2

## t) Force negative number of hours to 0

In [30]:
data_test.loc[data_test["hours_OBR-OBC"]<0,"hours_OBR-OBC"] = 0
data_test.loc[data_test["hours_OBC-OBV"]<0,"hours_OBC-OBV"] = 0
data_test.loc[data_test["hours_OBV-OB1Acc"]<0,"hours_OBV-OB1Acc"] = 0
data_test.loc[data_test["hours_OB1Acc-FTUc"]<0,"hours_OB1Acc-FTUc"] = 0

## u) Categorical variable encoding with OHE

In [31]:
enc = pickle.load(open('ohe.pkl', 'rb'))

In [32]:
var_ohe = ["canal","debit_account_bank","credit_account_bank","credit_account_type","lookup_key_type","prov_region"]
encoded_data = pd.DataFrame(enc.transform(data_test[var_ohe]).toarray(),columns=enc.get_feature_names(), index=data_test.index)

In [33]:
data_ready = data_test.drop(columns=var_ohe).join(encoded_data)

In [34]:
data_ready.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28816 entries, 00012104-7737-476c-b3c7-eb8d1452002b to ffff699c-980a-4a2a-9b63-ca8f844b3c4b
Data columns (total 50 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   transfer_between_own_accounts            28816 non-null  int32  
 1   actividad_principal                      28816 non-null  int64  
 2   cant_attempts                            28816 non-null  int64  
 3   gender                                   27832 non-null  float64
 4   age_group                                28635 non-null  float64
 5   cant_contactos_bancos                    28816 non-null  int64  
 6   cant_contactos_ank                       28816 non-null  int64  
 7   cantidad_de_cuentas                      28816 non-null  int64  
 8   cantidad_de_bancos                       28816 non-null  int64  
 9   cantidad_cvu                             28816 non-null

# 3) Load model

In [35]:
model = pickle.load(open('model_trained.pkl', 'rb'))

# 4) Predict

In [36]:
predictions = model.predict_proba(data_ready)[:,1]

In [37]:
low_prob_users = predictions[predictions<0.42]

In [38]:
high_prob_users = predictions[predictions>0.8]

In [39]:
predictions_with_user_id = pd.DataFrame(np.zeros(shape=(predictions.shape[0],3)), columns=['user_id','prob','classif'])

In [40]:
predictions_with_user_id.iloc[:,0] = data_ready.index
predictions_with_user_id.iloc[:,1] = predictions
predictions_with_user_id.loc[predictions_with_user_id['prob']<0.42,'classif'] = 'low_score'
predictions_with_user_id.loc[predictions_with_user_id['prob']>0.8,'classif'] = 'high_score'

In [42]:
predictions_with_user_id.to_csv('predictions.csv', index=False)